<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Emotion-Identification4/ViTModel_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import zipfile
import os

zip_file = 'FER2013.zip'  # Change filename if necessary
extract_path = '/content/FER2013/'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the extraction
print(os.listdir(extract_path))

In [ ]:
# List contents of train and test folders
print(os.listdir(extract_path + 'train'))
print(os.listdir(extract_path + 'test'))

In [ ]:
#Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
# Load Training Data
train_generator = train_datagen.flow_from_directory(
    os.path.join(extract_path, 'train'),
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

In [ ]:
# Load Validation Data
validation_generator = train_datagen.flow_from_directory(
    os.path.join(extract_path, 'train'),
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Load Test Data
test_generator = test_datagen.flow_from_directory(
    os.path.join(extract_path, 'test'),
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical'
)


In [ ]:
# Vision Transformer Model

def create_vit_model(input_shape=(48, 48, 3), num_classes=7):
    inputs = layers.Input(shape=input_shape)

    # Patch Extraction
    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Transformer Block 1
    transformer_layer = layers.MultiHeadAttention(num_heads=8, key_dim=64)
    x = transformer_layer(x, x)
    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    # Transformer Block 2
    transformer_layer2 = layers.MultiHeadAttention(num_heads=8, key_dim=64)
    x = transformer_layer2(x, x)
    x = layers.LayerNormalization()(x)

    # Fully Connected Layers
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Output Layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Create and Compile Model
model = create_vit_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Adjusted LR
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Summary
model.summary()

In [ ]:
# Train the Model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    verbose=1
)

In [ ]:
# Evaluate Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Confusion Matrix & Classification Report
test_predictions = model.predict(test_generator, verbose=1)
test_labels = test_generator.classes
predicted_labels = np.argmax(test_predictions, axis=1)

cm = confusion_matrix(test_labels, predicted_labels)
accuracy = accuracy_score(test_labels, predicted_labels)
report = classification_report(test_labels, predicted_labels, target_names=test_generator.class_indices.keys())

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.title("Confusion Matrix")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Display Evaluation Results
print("\n--- Evaluation Summary ---")
print(f"Test Accuracy: {accuracy:.4f}")
print("\n--- Classification Report ---\n", report)